# Carousel mechanism design
Project: *LuSEE-Night* <br>
System: *Antenna Carousel* <br>
Author: *Joe Silber* <br>
Institution: *Lawrence Berkeley National Lab (LBNL)* <br>
Date: *September 2022* <br>

Abstract: Assessment of carousel (a.k.a. "turntable") mechanism and torques for the LuSEE-Night lunar instrument. The radio antennas rotate on a platform. Key design parameters and options for the rotation mechanism are considered here.

## Preliminaries

### Units and coordinate systems
Except where otherwise specified...
- all units are in meters, kilograms, newtons, and seconds
- all inertias and positions are with respect to a global coordinate system, with Z axis concentric to rotation axis

### Python initializations and units

In [1]:
import math
import numpy as np
deg = '\u00b0'
tab = f'{" ":4}'
horiz_rule = ''.join(['-']*10)
Nm_per_ozin = 1 / 141.61193227806
Nm_per_inlb = Nm_per_ozin * 16
m_per_in = 0.0254 
radpersec_per_rpm = 2 * math.pi / 60  # i.e. rad/sec per rev/min
K_to_other = {'K': lambda T: T,
              'C': lambda T: T - 273.15,
              'F': lambda T: (T - 273.15)*9/5 + 32.0,
             }
N_per_lbf = 4.448
ksi_per_Pa = 1.450377377E-7
G_earth = 9.81  # m/s^2
G_moon = 0.166 * G_earth

### Basic system properties

In [2]:
n_ant = 4  # number of antennas
operating_range_deg = 180  # degrees, within which the turntable needs to be able to position
rotation_range_margin_deg = 5  # degrees, extra mechanical positioning range for antibacklash moves, etc
rotation_range_margin = math.radians(rotation_range_margin_deg)
rotation_range = [math.radians(operating_range_deg)/2 + rotation_range_margin,
                 -math.radians(operating_range_deg)/2 - rotation_range_margin]  # radians
total_rotation_range = max(rotation_range) - min(rotation_range)
print(f'Total rotation range = {total_rotation_range:.3f} rad = {math.degrees(total_rotation_range):.1f} deg')

Total rotation range = 3.316 rad = 190.0 deg


## Key mechanical design criteria
The design criteria below will be used in sizing components and ensuring sufficient mechanical factors of safey. These criteria are informed by key recommendations in NASA's [General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf) (accessed 2022-07-09).

### Structural load limit
The limit loads for the carousel mechanism are derived from GEVS Table 2.4-3,
*Generalized Random Vibration Test Levels*. For components weighing less than 22.7 kg, an overall qualification level (in $G_{rms}$) is directly specified. We then apply a proof factor per section 2.4.1.4.1, a peak acceleration factor, and a design factor to get the design limit. The design factor accommodates uncertainty in material properties and tolerances in the proof test method. These limits are applied for nominal accelerations in 3 orthogonal axes.

In [3]:
Grms_qual = 14.1  # per GEVs Table 2.4-3
proof_factor = 1.25  # per GEVs section 2.4.1.4.1
design_factor = 1.2  # covers uncertainties in matl props and test methods
peak_accel_factor = 5.0  # meant to represent ~ 5-sigma events
Grms_design = Grms_qual * proof_factor * design_factor
Gpeak_design = Grms_design * peak_accel_factor
print(f'Design mean load level = {Grms_design:.1f} G(rms)')
print(f'Design peak load level = {Gpeak_design:.1f} G')

Design mean load level = 21.1 G(rms)
Design peak load level = 105.8 G


### Torque margin
Torque margin ($TM$) for the carousel turntable is assessed per GEVS section 2.4.5.3. Two safety factors, $FS_k$ and $FS_v$ are applied to the "known" and "variable" torques in the system. GEVS states:

> The minimum available driving torque for the mechanism shall be determined based on the FS listed above. The Torque Margin (TM) shall be greater than zero and shall be calculated using the following formula:

$$TM = \frac{T_{avail}}{FS_k \sum T_{known} + FS_v \sum T_{variable}} - 1$$

In the present study, the available driving torque $T_{avail}$ is considered at output of the the gearmotor (which is the component whose performance requirements we are aiming to specify). The torques are defined in GEVS as:

> **Driving Torques**
>
> $T_{avail}$ = Minimum Available Torque or Force generated by the mechanism at worst case environmental conditions at any time in its life. If motors are used in the system, $T_{avail}$ shall be determined at the output of the motor, not including gear heads or gear trains at its output based on minimum supplied motor voltage. $T_{avail}$ similarly applies to other actuators such as springs, pyrotechnics, solenoids, heat actuated devices, etc.
> 
> **Resistive Torques**
>
> $\sum T_{known}$ = Sum of the fixed torques or forces that are known and quantifiable such as
accelerated inertias ($T = I \alpha$) and not influenced by friction, temperature, life, etc. A constant Safety Factor is applied to the calculated torque.
>
> $\sum T_{variable}$ = Sum of the torques or forces that may vary over environmental conditions and life such as static or dynamic friction, alignment effects, latching forces, wire harness
loads, damper drag, variations in lubricant effectiveness, including degradation or depletion of lubricant over life, etc.

Here we use the highest (PDR phase) torque safety factor values from GEVS:

In [4]:
FSk = 2.0  # 2.0 at preliminary design review phase
           # drops to 1.5 at critical design review
           # remains 1.5 at acceptance / qualification testing

FSv = 4.0  # 4.0 at preliminary design review phase
           # drops to 3.0 at critical design review
           # drops to 2.0 at acceptance / qualification testing

### Thermal criteria
The carousel system must survive the lunar day/night cycle. It must operate at intermediate temperatures around dusk/dawn.

The plots below, showing lunar temperature as a function of hour and latitude, were retrieved July 2022 from an article (exact reference?) found through citations [here](https://quickmap.lroc.asu.edu/layers):

![Lunar temperatures as function of hour and latitude](figures/lunar-temperatures.png)

We anticipate LuSEE-Night will land at one of two sites:

1. $155^o$ E and $15^o$ S
2. $175^o$ E and $22^o$ S

These indicate that the lander will experience temperature extrema of:

In [5]:
temp_ranges = {'extrema': [90, 390]}  # Kelvin

#### Survival temperature range
For design and testing purposes, the survival temperature range is obtained by increasing the expected lunar range by a margin on either end.

In [6]:
temp_margin = 10.0  # deg C
temp_ranges['survival'] = [min(temp_ranges['extrema']) - temp_margin,
                           max(temp_ranges['extrema']) + temp_margin]
max_temp_survival = max(temp_ranges['survival'])  # for less verbose usage below
min_temp_survival = min(temp_ranges['survival'])  # for less verbose usage below

#### Operating temperature range
The operating temperature for carousel rotation will be less extreme. We want a range which is moderate enough to includes practical motors, transmissions, and bearings, yet wide enough to allow a significant period in the dusk/dawn time window where ground-based operators can effectively send commands, monitor the response, and make any adjustments if necessary. A good operating temperature range also offers practical conditions for testing in the lab.

***? Under the assumption of a perfluoropolyether (PFPE) based lubricant in the gears and bearings ... ?***

***? For what temperature range do we have equipment on hand to practically test operation of the motors, bearings, transmissions---the full turntable assembly---in vacuum ?***

Considering these factors, we propose an operating temperature range of 243 K - 323 K, which should provide generous time windows ($\sim$ 1 lunar hour $\approx$ 30 earth hours each), at lunar dawn and dusk.

In [7]:
temp_ranges['operating'] = [243, 323]

#### Temperature ranges summary

In [8]:
for name, temp_range in temp_ranges.items():
    for unit, func in K_to_other.items():
        converted = [func(T) for T in temp_range]
        print(f'Temperature range = {min(converted):+4.0f} to {max(converted):+4.0f} {deg}{unit} ({name})')
    print('')

Temperature range =  +90 to +390 °K (extrema)
Temperature range = -183 to +117 °C (extrema)
Temperature range = -298 to +242 °F (extrema)

Temperature range =  +80 to +400 °K (survival)
Temperature range = -193 to +127 °C (survival)
Temperature range = -316 to +260 °F (survival)

Temperature range = +243 to +323 °K (operating)
Temperature range =  -30 to  +50 °C (operating)
Temperature range =  -22 to +122 °F (operating)



### General material properties

In [9]:
rho_Ti = 4430  # titanium density
rho_Al = 2700  # aluminum density
rho_BeCu = 8300  # beryllium copper density
rho_Cu = 8900  # copper density
rho_SS = 7800  # stainless steel density - note *martensitic* 440C stainless is used here, which is the hard material in stainless bearings from Kaydon (not austenitic)
rho_plastic = 1300  # generic plastic density
E_Cu = 117e9 # copper modulus of elasticity
E_plastic = 2.5e9 # generic plastic modulus of elasticity
E_BeCu = 130e9  # beryllium copper modulus of elasticity
E_Al = 70e9  # aluminum modulus of elasticity
E_SS = 215e9  # 440C stainless modulus of elasticity
E_Ti = 113.8e9  # Ti grade 5 stainless modulus of elasticity
nu_SS = 0.28  # 440C stainless poisson ratio
nu_Al = 0.33   # aluminum poisson ratio
nu_BeCu = 0.3  # beryllium copper poisson ratio
nu_PI = 0.41  # polyimide (Vespel SP1) poisson ratio
nu_Ti = 0.342  # # Ti grade 5 poisson ratio
CTE_Al = 23.6e-6  # aluminum coeff of thermal expansion per K
CTE_SS = 10.2e-6  # 440C stainless coeff of thermal expansion per K
CTE_PI = 50e-6  # approx CTE of polyimide per K
CTE_Ti = 8.6e-6  # approx CTE of grade 5 titanium per K
Sy_SS = 1660e6  # 440C yield strength, see comments below
Sy_Al_vs_temp = np.transpose([[-196, 324], [-80, 290], [-28, 283], [24, 276], [100, 262], [149, 214], [204, 103]])  # aluminum yield strength vs temperature, units (degC, MPa), see comments below
Sy_Al = lambda T: np.interp(T, Sy_Al_vs_temp[0]+273, Sy_Al_vs_temp[1]*1e6)  # interpolation function for yield strength (Pa) vs temperature (K)
Sy_Ti_Gr9annealed = 517e6  # grade 9 annealed titanium yield stress
for T in [min_temp_survival, max_temp_survival]:
    print(f'Yield strength for aluminum interpolated at temperature {T:3.0f} K --> {Sy_Al(T)/1e6:.1f} MPa = {Sy_Al(T)*ksi_per_Pa:.1f} ksi')

Yield strength for aluminum interpolated at temperature  80 K --> 323.1 MPa = 46.9 ksi
Yield strength for aluminum interpolated at temperature 400 K --> 235.6 MPa = 34.2 ksi


### Strength comments
Aluminum alloy and temper is assumed to be 6061-T651, for which yield strength varies from 324 MPa @ -196°C to 214 MPa @ +149°C.

Annealed yield strength of 440C martensitic stainless steel is [448 MPa](https://www.pennstainless.com/resources/product-information/stainless-grades/400-series/440c-stainless-steel), but certainly it will have been hardened for use in bearings. Hardening will be something more on the order of 1660-1900 MPa, depending on the tempering temperature (1900 MPa @ 204°C to 1660 MPa @ 371°C). Here we assume the low end of the tempered yield strength will be the minimum.

It is well known that the toughness of martensitic alloys is much reduced at cryogenic temperatures (below the martensitic transition point). However in our use case of a bearing, there will be no loads other than the thermal contraction of the turntable plate toward the bearing (assuming an aluminum plate), which will apply uniform compression to the bearing, opposing rather than promoting any potential for crack growth.

### Carbon fiber properties

In [10]:
rho_f = 2170  # carbon fiber density, e.g. CN-80
rho_m = 1170  # resin matrix density, e.g. EX-1515
Mm = 0.32  # matrix mass fraction, i.e. prepreg resin content from vendor
Vv = 0.005  # as-cured void fraction
Vf = (1 - Vv) / (1 + rho_f / rho_m * Mm / (1 - Mm))  # fiber volume fraction, assuming no bleed (conservative)
cfrp_density = rho_f * Vf + rho_m * (1 - Vf - Vv)
print(f'Estimated carbon fiber reinforced plastic (CFRP) density = {cfrp_density:.1f} kg/m^3, volume fraction = {Vf*100:.1f}%')

Estimated carbon fiber reinforced plastic (CFRP) density = 1695.4 kg/m^3, volume fraction = 53.1%


## Loads
Total mass budget for the carousel system is:

In [11]:
mass_budget_total = 7.8  # kg

Some fraction of this mass (in particular the drive motor and transmission) will be static.

For the purposes of this document, we subdivide the rotating payload into the following estimated masses, moments, and transmission ratios.

### Turntable plate
Three common engineering materials are being considered for the turntable platter: aluminum, titanium, or carbon fiber reinforced plastic (CFRP).

In [12]:
plate_od = 0.400  # outer diameter
plate_id = 0.079  # inner diameter
plate_t = 0.010  # thickness
plate_z = 0.016  # position along bearing axis of plate c.g.
plate_r2, plate_r1 = plate_od/2, plate_id/2  # radii
plate_area = math.pi * (plate_r2**2 - plate_r1**2)
uniform_plate_inertia_over_mass = 0.5 * (plate_r1**2 + plate_r2**2)

Bending stiffness is achieved either through a honeycomb core or ribs (of which a nominal design might look like that illustrated here:
![Nominal ribs for a turntable](figures/nominal-turntable-ribs.png)

In [13]:
core_vf = 0.15 # honeycomb core volume fraction, typically about 8-32% depending on design
ribs_af = 0.20 # approx area fraction of some nominal ribs stiffening design for turntable plate

#### CFRP sandwich construction
CFRP would have a higher stiffness per unit mass than the metals, as well as some natural structural damping. We would likely use an intermediate modulus fiber in a quasi-isotropic, balanced, symmetric layup. Matrix would be 350$^o$F (450 K) cured resin such as Toray RS-3C, commonly used in spacecraft. The design would need to be sufficiently low-strain such that the total temperature change from processing (450 K) down to lunar night (100 K) does not exceed fiber strain limits. A honeycomb sandwich construction would be most mass-efficient, with titanium or aluminum inserts at bolted connections. Thermal expansion effects would be low, several parts per million (ppm) per degree K. Low out-of-plane thermal conductivity would be of some benefit in reducing the heat transfer rate from the turntable to the rest of the spacecraft below it. The main detractions of CFRP are the higher design and fabrication costs.

In [14]:
plate_sandwich_skin_t = 0.0016
plate_sandwich_core_t = plate_t - 2 * plate_sandwich_skin_t
plate_sandwich_skin_rho = cfrp_density
plate_sandwich_core_rho = rho_Al * core_vf
plate_sandwich_mass = (2*plate_sandwich_skin_rho*plate_sandwich_skin_t + plate_sandwich_core_rho*plate_sandwich_core_t) * plate_area
plate_masses = {'CFRP sandwich': plate_sandwich_mass}

#### Metal ribbed plate construction
Material would be either titanium or aluminum.

A machined aluminum plate would be the least expensive to fabricate. Detractions are the higher CTE ($\sim$12-25 ppm/K over the 100K-400K temperature range) and reduced stress allowables at sustained high temperature (7075-T651 aluminum, for example, may have yield strength reduction from $\sim$ 500 MPa at room temperature to $\sim$ 180 MPa when sustained at 150$^o$C (423 K). The high thermal conductivity of aluminum allows faster heat transfer toward the spacecraft. All these detractions can likely be mitigated with use of appropriate design allowables and thermal breaks.

It is noted that the higher temperatures would only be experienced by the plate at the moon, where gravity loads are 1/6.

Grade 5 titanium would be a robust solution. Material cost would be comparable to CFRP, though fabrication complexity would be simpler. CTE is relatively low and stable ($\sim$5-9 ppm/K over range 100K-400K), strength is high, and the very low thermal conductivity ($\sim$5-8 W/m*K) would provide some natural insulation of the rest of the spacecraft from the exposed turntable's temperature. For cost and ease of manufacturing we would probably not machine the plate out of thick titanium, but rather screw together a thinner face sheet with stiffening ribs.

In [15]:
plate_rhos = {'aluminum': rho_Al,
              'titanium': rho_Ti,
             }  # density
plate_face_t = 0.0015  # face plate thickness
plate_ribs_t = plate_t - plate_face_t
plate_volume = (plate_face_t + plate_ribs_t * ribs_af) * plate_area
for name, rho in plate_rhos.items():
    plate_masses[name] = rho * plate_volume

#### Plate construction options summary

In [16]:
plate_inertias = {}
for name, mass in plate_masses.items():
    plate_inertias[name] = mass * uniform_plate_inertia_over_mass  # note this uniform assumption is good for a typical evenly-spaced rib pattern
    print(f'Turntable mass = {plate_masses[name]:.3f} kg, inertia = {plate_inertias[name]:.4f} kg/m^2 ({name})')

Turntable mass = 0.988 kg, inertia = 0.0205 kg/m^2 (CFRP sandwich)
Turntable mass = 1.043 kg, inertia = 0.0217 kg/m^2 (aluminum)
Turntable mass = 1.712 kg, inertia = 0.0356 kg/m^2 (titanium)


In [17]:
# Select material for this study.
plate_material = 'aluminum'
plate_mass = plate_masses[plate_material]
plate_inertia = plate_inertias[plate_material]
print(f'Selected the {plate_material} option for this study.')

Selected the aluminum option for this study.


### Antennas
The four antennas will be stacer tubes, equally spaced around the turntable. Deployed length is 3 m for all antennas. Deployment angle is slightly above horizontal, counteracting gravity sag. Here this slight bend is neglected, and treated as purely horizontal.

In [18]:
boom_deployed_wall_in = 0.005  # inches, antenna approx wall thickness when deployed
boom_deployed_diam_in = 0.35  # inches, antenna approx diameter when deployed
boom_wall, boom_diam, boom_area, boom_length, boom_inertia = {}, {}, {}, {}, {}
boom_wall['deployed'] = boom_deployed_wall_in * m_per_in
boom_diam['deployed'] = boom_deployed_diam_in * m_per_in
boom_area['deployed'] = math.pi * boom_diam['deployed'] * boom_wall['deployed']
boom_length['deployed'] = 3.0
boom_volume = boom_length['deployed'] * boom_area['deployed']
boom_rho = rho_BeCu  # select material for antenna
boom_mass = boom_volume * boom_rho
print(f'Antenna boom mass = {boom_mass:.3f} kg each')

Antenna boom mass = 0.088 kg each


In [19]:
boom_length['stowed'] = 0.105  # assumption based on July 2022 CAD model
boom_area['stowed'] = boom_volume / boom_length['stowed']
boom_diam['stowed'] = boom_diam['deployed']  # approx guess
boom_wall['stowed'] = boom_area['stowed'] / (math.pi * boom_diam['stowed'])
boom_z = 0.077  # approx position along bearing axis of boom c.g.

Each antenna also has a tip piece. The mass and size are estimated here from the July 2022 CAD model.

In [20]:
ant_tip_mass = 0.029
ant_tip_length = 0.156
ant_tip_diam = boom_diam['deployed'] - boom_wall['deployed']
ant_tip_inertia_about_cg = ant_tip_mass/48 * (3*ant_tip_diam**2 + 4*ant_tip_length**2)
ant_tip_z = boom_z  # approx position along bearing axis of antenna tip c.g. (deflected)
print(f'Antenna mass (boom + tip) = {boom_mass + ant_tip_mass:.3f} kg each')

Antenna mass (boom + tip) = 0.117 kg each


In [21]:
boom_root_radial_position = plate_od/2 - boom_length['stowed'] # where the antenna base end is, radially on the turntable
ant_inertia = {}
for k in ('deployed', 'stowed'):
    boom_OD = boom_diam[k] + boom_wall[k] / 2
    boom_ID = boom_diam[k] - boom_wall[k] / 2
    boom_inertia_about_cg = boom_mass/48 * (3*boom_OD**2 + 3*boom_ID**2 + 4*boom_length[k]**2)
    boom_offset = boom_root_radial_position + boom_length[k] / 2
    ant_tip_offset = boom_root_radial_position + boom_length[k] - ant_tip_length/2
    boom_inertia = boom_inertia_about_cg + boom_mass * boom_offset**2
    ant_tip_inertia = ant_tip_inertia_about_cg + ant_tip_mass * ant_tip_offset**2
    ant_inertia[k] = boom_inertia + ant_tip_inertia
    print(f'Antenna inertia (boom + tip) = {ant_inertia[k]:.4f} kg/m^2 each ({k})')

Antenna inertia (boom + tip) = 0.5550 kg/m^2 each (deployed)
Antenna inertia (boom + tip) = 0.0025 kg/m^2 each (stowed)


### Antenna deployers
As of July 2022, the deployers and attached hardware (including pre-amp?) are assumed to be $\sim$ 0.75 kg per antenna. This value will change somewhat as the design is refined.

For the purposes of this calculation, the deployer geometry is treated as a cuboid with side lengths estimated based on the stowed antenna length.

In [22]:
deployer_mass = 0.75
deployer_z = 0.070  # position along bearing axis of deployer c.g.
deployer_length = boom_length['stowed'] * 3
deployer_width = boom_length['stowed']
deployer_radial_position = plate_od / 2
deployer_inertia_about_cg = deployer_mass / 12 * (deployer_width**2 + deployer_length**2)
deployer_inertia = deployer_inertia_about_cg + deployer_mass * deployer_radial_position**2
print(f'Deployer mass = {deployer_mass:.3f} kg (each)')
print(f'Deployer inertia = {deployer_inertia:.4} kg/m^2 (each)')

Deployer mass = 0.750 kg (each)
Deployer inertia = 0.03689 kg/m^2 (each)


### Central bearing
The central bearing and its associated clamping hardware are approximated as a solid tube of metal.

In [23]:
central_od = 0.090  # outer diameter
central_id = 0.070  # inner diameter
central_length = 0.020  # length
central_r1, central_r2 = central_id/2, central_od/2  # radii
central_rho = rho_SS
central_mass = central_rho * central_length * math.pi * (central_r2**2 - central_r1**2)
central_inertia = 0.5 * central_mass * (central_r1**2 + central_r2**2)
print(f'Central bearing mass = {central_mass:.3f} kg')
print(f'Central bearing inertia = {central_inertia:.4} kg/m^2')

Central bearing mass = 0.392 kg
Central bearing inertia = 0.0006371 kg/m^2


### Transmission
The motor will be mounted off-axis to the central bearing, connected via a titanium drive shaft, axially flexible coupling, and a universal joint.

In [24]:
transmission_efficiency = 0.90  # assumed value, probably conservative, considering friction in u-joint
flex_coupling_mass = 0.
ujoint_mass = 0.
shaft_od = 0.5 * m_per_in  # outer diameter
shaft_wall = 0.026 * m_per_in  # wall thickness
shaft_id = shaft_od - 2 * shaft_wall  # inner diameter
shaft_J = math.pi / 2 * ((shaft_od/2)**4 - (shaft_id/2)**4)  # second moment of area
shaft_yield_stress = Sy_Ti_Gr9annealed
shear_yield_stress = shaft_yield_stress / 3**0.5  # von Mises criterion: 1/3**0.5, Tresca: 1/2
shaft_yield_torque = shear_yield_stress * shaft_J / (shaft_od/2)
print(f'For drive shaft with...'
      f'\n{tab}... outer diameter = {shaft_od*1000:.2f} mm'
      f'\n{tab}... inner diameter = {shaft_id*1000:.2f} mm'
      f'\n{tab}... yield stress = {shaft_yield_stress/1e6:.1f} MPa = {shaft_yield_stress*ksi_per_Pa:.1f} ksi'
      f'\n --> yield torque = {shaft_yield_torque:.1f} N*m = {shaft_yield_torque/Nm_per_inlb:.1f} in*lb')

For drive shaft with...
    ... outer diameter = 12.70 mm
    ... inner diameter = 11.38 mm
    ... yield stress = 517.0 MPa = 75.0 ksi
 --> yield torque = 42.7 N*m = 377.7 in*lb


### Motor shaft, motor gearhead, and motor bearing
In the present study (July 2022), whose initial purpose is to define specifications for the motor, the internal details of the motor are not considered. They could be added to this study at a later date, after we have selected a specific motor model.

### External dust seal

Some dust seal(s) will be required to prevent lunar dust from accessing the bearing and transmission. These impose some friction load on the assembly.

Here I assume the seal contact is made against a flat face by annular, thin-walled polyimide. In this approach, there would be one seal at the turntable rotation axis, and another at the motor output, covering the off-axis transmission components. (The motor may additionally have internal seals in its bearing at the output shaft, TBD.)

The seal at the central axis will bear either on the rotating turntable, or else be mounted to the turntable and bear on its opposing counterpart (i.e. the top plate or the central shaft support). Preload force will be relatively low, since only dust needs to be excluded.

Some comments are offered in another section about the option of alternatively, or additionally, having internal seals integral to the bearing. There are both advantages and disadvantages. It would be necessary for the bearing vendor to fabricate using a seal material with which we are comfortable at the lunar temperature extremes and in vacuum.

In [25]:
seal_contact_radius = 0.080  # annulus of contact for the seals
seal_contact_width = 0.003  # rough guess as to desirable contact width, will depend on seal design details
seal_contact_pressure_millibar = 5  # rough guess as to desirable clamping pressure (per seal)
seal_contact_pressure = seal_contact_pressure_millibar * 1000
seal_contact_area = 2 * math.pi * seal_contact_radius * seal_contact_width
seal_clamping_force = seal_contact_pressure * seal_contact_area
print(f'External dust seal at radius {seal_contact_radius:.3f} m: clamping force = {seal_clamping_force:.1f} N')

External dust seal at radius 0.080 m: clamping force = 7.5 N


Datasheet values for the static and dynamic friction coefficient of [Vespel SP-1](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-1) are listed at 0.29 and 0.35, respectively. I presume a fixed, static value of 0.35 for the purposes of this calculation, and estimate a fixed torque based on the assumed contact annulus.

In [26]:
seal_mu = 0.35
seal_torque = seal_mu * seal_contact_radius * seal_clamping_force
print(f'External dust seal static torque = {seal_torque:.3f} N*m = {seal_torque / Nm_per_ozin:.1f} oz*in')

External dust seal static torque = 0.211 N*m = 29.9 oz*in


### Bearing friction
For the purposes of preliminary overall system torque calculations, I presume a bearing with its own integrated dust seal. This may not be the case, as discussed in more detail in the bearing selection section below. But presently, in sizing the motor and transmission, I assume selection of a single Type-X, nitrile double-sealed, stainless steel bearing, e.g. Kaydon WA025XP0. The vendor provides a static torque value for this bearing, given default lubrication and temperature conditions:

In [27]:
bearing_torque_ozin = 8.0  # oz*in
bearing_torque = bearing_torque_ozin * Nm_per_ozin
print(f'Central bearing static torque = {bearing_torque:.3f} N*m')

Central bearing static torque = 0.056 N*m


### Harness
The primary elastic contributor to motor driving torque is the cable harness. Its torsional stiffness is estimated here by counting the number of conductors, assuming a diameter for each conductor, and then adding up their torsional stiffness plus their insulating coatings in parallel. The individual conductors are here treated as solid core, though in reality they would be stranded.

In [28]:
wires_per_antenna = 20
n_wires = n_ant * wires_per_antenna
wire_diam_mm = 0.35
wire_insul_wall_mm = 0.15  # wall thickness of insulator on wire
wire_diam = wire_diam_mm / 1000
wire_insul_wall = wire_insul_wall_mm / 1000
J_wire = math.pi/32 * wire_diam**4  # wire polar moment of inertia
J_insul = math.pi/32 * ((wire_diam + 2*wire_insul_wall)**4 - wire_diam**4)  # insulation polar moment of inertia
harness_length = 0.3  # estimated from July 2022 CAD model
shear_modulus = lambda E: E / (2*(1 + 0.3))  # approx shear modulus for typical poisson ratio
insulated_wire_stiffness = (shear_modulus(E_Cu) * J_wire + shear_modulus(E_plastic) * J_insul) / harness_length  # N*m / rad
harness_stiffness = n_wires * insulated_wire_stiffness
harness_max_torque = harness_stiffness * max(rotation_range)
for unit, scale in {'N*m/rad': 1.0, 'N*mm/deg': 180/math.pi*1000, 'oz*in/deg': 180/math.pi/Nm_per_ozin}.items():
    print(f'Harness approx torsional stiffness = {harness_stiffness*scale:.4g} {unit}')
for unit, scale in {'N*m': 1.0, 'N*mm': 1000., 'oz*in': 1/Nm_per_ozin}.items():
    print(f'Harness max torque resistance = {harness_max_torque*scale:.4g} {unit}')

Harness approx torsional stiffness = 0.02179 N*m/rad
Harness approx torsional stiffness = 1249 N*mm/deg
Harness approx torsional stiffness = 176.8 oz*in/deg
Harness max torque resistance = 0.03614 N*m
Harness max torque resistance = 36.14 N*mm
Harness max torque resistance = 5.117 oz*in


### Gear inertia
The vendor datasheet for Globe Motors A-1430, as an example, indicates a maximum gear inertia. While included here for completeness, in practice this value is negligible with respect to the overall system. (This fact is displayed further down, see printout in the "Known" torques calculation below.)

In [29]:
max_gear_inertia_ozinsec2 = 1.8e-6  # oz*in*sec^2, Globe A-1430
max_gear_inertia = max_gear_inertia_ozinsec2 * Nm_per_ozin  # kg*m^2

## Performance
Our needs for rotational speed and acceleration are relatively low. 

### Speed
Repositioning will take on the order of minutes. The constraints imposed here are generally based on rough practical assumptions. There are no strong constraints at either the too-slow or too-fast ends of this range.

In [30]:
min_slew_time = 2.  # when going the maximum slew distance
max_slew_time = 300.  # when going the maximum slew distance
antibacklash_move_deg = 3.0
antibacklash_move = math.radians(antibacklash_move_deg)
max_slew_distance = total_rotation_range + antibacklash_move
min_allowable_speed = max_slew_distance / (max_slew_time)
max_allowable_speed = max_slew_distance / (min_slew_time)
print(f'Min allowable speed = {min_allowable_speed/radpersec_per_rpm:.3g} rpm = {math.degrees(min_allowable_speed):.1f} deg/s.')
print(f'Max allowable speed = {max_allowable_speed/radpersec_per_rpm:.3g} rpm = {math.degrees(max_allowable_speed):.1f} deg/s.')

Min allowable speed = 0.107 rpm = 0.6 deg/s.
Max allowable speed = 16.1 rpm = 96.5 deg/s.


### Acceleration
Acceleration must not exceed the buckling strength of the extended antenna. Here we estimate the buckling critical load per Roark's 7th Edition:

![Transverse buckling of thin-walled tube](figures/roarks_7th_tube_transverse_buckling.png)

In [31]:
buckling_design_safety_factor = 100  # stay conservative, since this is both Euler instability and since inertias / motor details not yet fully defined
tube_transverse_buckling_K = 0.72  # using minimum value here, to stay on conservative side
boom_root_radial_position = plate_od
boom_critical_moment = tube_transverse_buckling_K * E_BeCu / (1 - nu_BeCu**2) * boom_length['deployed'] * boom_wall['deployed']**2
boom_allowable_moment = boom_critical_moment / buckling_design_safety_factor
boom_allowable_accel = boom_allowable_moment / ant_inertia['deployed']
boom_allowable_accel_period = max_allowable_speed / boom_allowable_accel
print(f'Antenna critical buckling moment = {boom_critical_moment:.1f} N*m')
print(f'Applying safety factor of {buckling_design_safety_factor} --> allowable moment = {boom_allowable_moment:.1f} N*m')
print(f'--> max allowable acceleration = {boom_allowable_accel:.1f} rad/s^2 = {math.degrees(boom_allowable_accel):.1f} deg/s^2 = {boom_allowable_accel/(2*math.pi):.1f} rev/s^2')
print(f'--> min allowable period of acceleration = {boom_allowable_accel_period:.3g} sec')

Antenna critical buckling moment = 4976.9 N*m
Applying safety factor of 100 --> allowable moment = 49.8 N*m
--> max allowable acceleration = 89.7 rad/s^2 = 5138.4 deg/s^2 = 14.3 rev/s^2
--> min allowable period of acceleration = 0.0188 sec


The boom buckling-limited acceleration is indeed many orders of magnitude faster than we expect any practical real-world system to perform, and hence is ignored for the remainder of the calculations.

We assert limits on nominal acceleration based on the margin of extra mechanical range we are mechanically providing the turntable. We prefer the system to be capable of fully accelerating within about twice that angular distance. This is not a hard physical constraint, just a practical consideration.

In [32]:
accel_dist = rotation_range_margin * 2.0
nom_accel = [v**2 / (2 * accel_dist) for v in [min_allowable_speed, max_allowable_speed]]
print(f'Nominal bounds on system acceleration rate = ({min(nom_accel):.3g}, {max(nom_accel):.3g}) rad/s^2 = ({math.degrees(min(nom_accel)):.3g}, {math.degrees(max(nom_accel)):.1f}) {deg}/s^2')
assert max(nom_accel) <= boom_allowable_accel, f'max nominal acceleration is higher than boom can tolerate'

Nominal bounds on system acceleration rate = (0.000361, 8.13) rad/s^2 = (0.0207, 465.6) °/s^2


## Torque margin calculation

### Resistive known torques

In [33]:
total_inertia = plate_inertia + n_ant * (ant_inertia['deployed'] + deployer_inertia) + central_inertia + max_gear_inertia
T_known_range = [total_inertia * a for a in nom_accel]
T_known_range_ozin = [T / Nm_per_ozin for T in T_known_range]
print(f'Total payload inertia (i.e. not including motor) = {total_inertia:.4g} kg*m^2')
print(f'(Note: max gear inertia {max_gear_inertia:.3g} kg*m^2 represents a fraction {max_gear_inertia/total_inertia:.2g} of total.)')
print(f'Possible range for total "known" resistive torque @ min accel = {min(T_known_range):.3g} N*m = {min(T_known_range_ozin):.3g} oz*in')
print(f'                                              ... @ max accel = {max(T_known_range):.3g} N*m = {max(T_known_range_ozin):.3g} oz*in')

# Select a single nominal T_known
# Note that GEVS torque margins, applied later below, will add significant torque to the system.
# In other words, this is not the place to apply large margin.
T_known_margin = 0.01  # small, arbitrary fraction between the (wide-ranging) min and max at which to set nominal T_known
T_known = T_known_margin * (max(T_known_range) - min(T_known_range)) + min(T_known_range)
T_known_ozin = T_known / Nm_per_ozin
print(f'\nNominal design value selected for total "known" resistive torque = {T_known:.3g} N*m = {T_known_ozin:.3g} oz*in')

Total payload inertia (i.e. not including motor) = 2.39 kg*m^2
(Note: max gear inertia 1.27e-08 kg*m^2 represents a fraction 5.3e-09 of total.)
Possible range for total "known" resistive torque @ min accel = 0.000863 N*m = 0.122 oz*in
                                              ... @ max accel = 19.4 N*m = 2.75e+03 oz*in

Nominal design value selected for total "known" resistive torque = 0.195 N*m = 27.6 oz*in


### Resistive variable torques

In [46]:
T_static_friction = [bearing_torque, seal_torque + bearing_torque]  # range of options, since final sealing config not yet determined
T_friction_transmission_loss = [T * (1 - transmission_efficiency) for T in T_static_friction]
T_known_transmission_loss = (1 - transmission_efficiency) * T_known
T_transmission_loss = [T_known_transmission_loss + T for T in T_friction_transmission_loss]
T_variable = [min(T_static_friction) + min(T_transmission_loss),
              max(T_static_friction) + max(T_transmission_loss) + harness_max_torque]
print(f'Range for static friction torque = ({min(T_static_friction):.3f}, {max(T_static_friction):.3f}) N*m = ({min(T_static_friction)/Nm_per_ozin:.1f}, {max(T_static_friction)/Nm_per_ozin:.1f}) oz*in')
print(f'Range for transmission loss = ({min(T_transmission_loss):.3f}, {max(T_transmission_loss):.3f}) N*m = ({min(T_transmission_loss)/Nm_per_ozin:.1f}, {max(T_transmission_loss)/Nm_per_ozin:.1f}) oz*in')
print(f'Range for "variable" resistive torque = ({min(T_variable):.3f}, {max(T_variable):.3f}) N*m = ({min(T_variable)/Nm_per_ozin:.1f}, {max(T_variable)/Nm_per_ozin:.1f}) oz*in')

Range for static friction torque = (0.056, 0.268) N*m = (8.0, 37.9) oz*in
Range for transmission loss = (0.025, 0.046) N*m = (3.6, 6.6) oz*in
Range for "variable" resistive torque = (0.082, 0.350) N*m = (11.6, 49.6) oz*in


### Minimum driving torque
We use the resistive known and variable torques, in combination with the GEVS safety factors, to determine the minimum allowable motor torque.

In [35]:
TM = 0.0  # set to 0 for minimum allowable, i.e. positive torque margin
T_avail_min = (TM + 1) * (FSk * T_known + FSv * max(T_variable))
print(f'For positive torque margin, with safety factors FSk = {FSk} and FSv = {FSv},\n'
      f'minimum gearmotor torque shall be T_avail = {T_avail_min:.3g} N*m = {T_avail_min/Nm_per_ozin:.3g} oz*in')

For positive torque margin, with safety factors FSk = 2.0 and FSv = 4.0,
minimum gearmotor torque shall be T_avail = 1.79 N*m = 254 oz*in


### Maximum driving torque (buckling constraint)
As discussed above, the acceleration rate to cause antenna boom buckling is many orders of magnitude outside what any practical motor in this application will achieve. This is confirmed below, by a simple scaling argument:

In [36]:
scaled_buckling_torque = boom_allowable_accel / (T_known / total_inertia) + min(T_variable)
print(f'To even approach the boom buckling safety factor {buckling_design_safety_factor}, available motor driving'
      f'\ntorque would have to be >= {scaled_buckling_torque:.1f} N*m = {scaled_buckling_torque/Nm_per_ozin:.1f} oz*in')

To even approach the boom buckling safety factor 100, available motor driving
torque would have to be >= 1098.8 N*m = 155607.8 oz*in


## Central bearing loads

Loads are evaluated in three dimensions (radial, thrust, and moment) independently. The independence assumption is based on the presumption that our very high and conservative design loading levels (e.g. the peak acceleration allowables above) would only be encountered essentially monodirectional, one time, brief events.

Additional conservatism in the mass estimates above carry into these loads.

In [62]:
bearing_supported_mass = plate_mass + n_ant * (boom_mass + ant_tip_mass + deployer_mass)
bearing_supported_moment = plate_mass*plate_z + n_ant * (boom_mass*boom_z + ant_tip_mass*ant_tip_z + deployer_mass*deployer_z)
bearing_loads_stationary = {'thrust': bearing_supported_mass,  # nominally upright
                            'radial': bearing_supported_mass,  # when tipped 90 deg
                            'moment': bearing_supported_moment,  # when tipped 90 deg
                           }
bearing_design_loads = {'rms': {name: load * Grms_design * G_earth for name, load in bearing_loads_stationary.items()},
                        'peak': {name: load * Gpeak_design * G_earth for name, load in bearing_loads_stationary.items()},
                       }
Fr = bearing_design_loads['peak']['radial']
Fa = bearing_design_loads['peak']['thrust']
single_bearing_diam = 0.080  # assumed pitch circle diameter of single bearing
duplex_bearing_sep = 0.020  # assumed axial separation distance between duplexed bearings (where applicable)
Fm_single = bearing_design_loads['peak']['moment'] / (single_bearing_diam/2)
Fm_duplex = bearing_design_loads['peak']['moment'] / duplex_bearing_sep
n_balls_typ = 24  # approximate typical number of balls, will vary among models
d_ball_typ = 0.006  # approximate typical diameter of balls, will vary among models
ZD2 = n_balls_typ * d_ball_typ**2
FaZD2_lbperin2 = (Fa / N_per_lbf) / (ZD2 / m_per_in**2)
bearing_factors = {'radial deep groove': {'X': 0.56, 'Y': 1.04}, # per NHBB catalog
                       f'angular contact (5{deg})': {'X': 0.57, 'Y': 0.93},  # per SKF (contact angle 40°), https://evolution.skf.com/us/four-point-contact-ball-bearings-two-in-one
                       f'angular contact (10{deg})': {'X': 0.46, 'Y': 1.01},
                       f'angular contact (15{deg})': {'X': 0.44, 'Y': 1.00},
                       f'angular contact (20{deg})': {'X': 0.43, 'Y': 1.00},
                       'four-point contact': {'X': 0.60, 'Y': 1.07},  # per SKF (contact angle 35°), https://evolution.skf.com/us/four-point-contact-ball-bearings-two-in-one
                      }
P = 0,60×Fr + 1,07×Fa
for a four-point contact ball bearing (contact angle 35°)

P = 0,57×Fr + 0,93×Fa
for a matched set of angular contact ball bearings (contact angle 40°)

print('Fa / (Z * D^2) ~ {FaZD2_lbperin2:.1f} lbf/in^2, hence here using'
      '\napproximations found in NHBB / Minebea catalog for 750 lbf/in^2 case:')
for key, XY in bearing_factors.items():
    print(f'{tab}{key:<21} ... X = {XY["X"]:.2f}, Y = {XY["Y"]:.2f}')

# note below that the loads here are really due to a sudden acceleration in one direction
# hence depending on the bearing type, it matters whether this is a sudden horizontal vs vertical acceleration
# this is handled in the maxima usages below
bearing_equiv_loads = {'crossed roller': max(Fr + Fm_single, 0.44*Fa),  # IKO catalog shows sum of all three terms
                       'radial deep groove': 0,  # per NHBB catalog
                       f'angular contact (5{deg})':0,
                       f'angular contact (10{deg})':0,
                       f'angular contact (15{deg})':0,
                       'four-point contact':0,
                      }
bearing_loads_str = f'Payload mass of {bearing_supported_mass:.2f} kg yields the following bearing\ndesign loads (these include all safety factors):'
for kind, loads in bearing_design_loads.items():
    bearing_loads_str += f'\n{tab}{kind} radial = {loads["radial"]:.0f} N = {loads["radial"]/N_per_lbf:.0f} lbf'
    bearing_loads_str += f'\n{tab}{kind} thrust = {loads["thrust"]:.0f} N = {loads["thrust"]/N_per_lbf:.0f} lbf'
    bearing_loads_str += f'\n{tab}{kind} moment = {loads["moment"]:.0f} N*m = {loads["moment"]/Nm_per_inlb:.0f} in*lb'

    #bearing_loads_str += f'\n{tab}static equivlent load = {
print(bearing_loads_str)
bearing_equiv_loads


Fa / (Z * D^2) ~ {FaZD2_lbperin2:.1f} lbf/in^2, hence here using
approximations found in NHBB / Minebea catalog for 750 lbf/in^2 case:
    radial deep groove    ... X = 0.56, Y = 1.04
    angular contact (5°)  ... X = 0.56, Y = 1.04
    angular contact (10°) ... X = 0.46, Y = 1.01
    angular contact (15°) ... X = 0.44, Y = 1.00
    angular contact (20°) ... X = 0.43, Y = 1.00


TypeError: 'int' object is not subscriptable

## Bearing mounting stress
The simplest bearing mounting methods would be relatively rigid, i.e. an interference fit or glue joint. Thermal contraction forces need to be checked to ensure no risk of either yielding the aluminum or damaging the bearing balls or races.

In [38]:
assembly_temperature = 293  # Kelvin
deltaT = assembly_temperature - min_temp_survival
differential_strain = (CTE_Al - CTE_SS) * deltaT  # roughly treated here as constant and linear, of course should be the integrated cryogenic strain
a = 3.5 * m_per_in  # bearing ID (solid ring approximation, conservative)
b = 4.0 * m_per_in  # bearing OD
c = b + 0.2  # approx equiv aluminum diameter surrounding the bearing
geom_factor_housing = (c**2 + b**2) / (c**2 - b**2)
geom_factor_bearing = (b**2 + a**2) / (b**2 - a**2)
p = differential_strain / ( (geom_factor_housing + nu_Al)/E_Al + (geom_factor_bearing - nu_SS)/E_SS )  # interface pressure, c.f. Shigley
print(f'Aluminum housing / stainless bearing interference pressure at {min_temp_survival} K = {p/1e6:.1f} MPa = {p*ksi_per_Pa:.1f} ksi')

hoop_stress = {'housing': p * geom_factor_housing,
               'bearing': -p * geom_factor_bearing,
              }
radial_stress = -p
equiv_stress = {}
for key, hoop in hoop_stress.items():
    equiv_stress = (radial_stress**2 + hoop**2 - radial_stress*hoop)**0.5
    yield_stress = Sy_Al(min_temp_survival) if key == 'housing' else Sy_SS
    print(f'{tab}{key} equivalent stress = {equiv_stress/1e6:.1f} MPa = {equiv_stress*ksi_per_Pa:.1f} ksi --> Yield stress FOS = {yield_stress / equiv_stress:.2f}')

Aluminum housing / stainless bearing interference pressure at 80.0 K = 50.6 MPa = 7.3 ksi
    housing equivalent stress = 99.1 MPa = 14.4 ksi --> Yield stress FOS = 3.26
    bearing equivalent stress = 358.7 MPa = 52.0 ksi --> Yield stress FOS = 4.63


As one increases the "approximate equivalent diameter" of the aluminum housing used in this first-order calculation, the aluminum stress decreases while the bearing stress increases. The true stiffness behavior of the housing will depend on ribbing details, and will be ultimately assessed more precisely in a finite element thermal model.

***TO-DO: Bearing balls contact forces in races***

## Launch lock
***TO-DO***

Analyze pin puller tension = mu x N, with N applied to a spring element that resists rotational vibration of plate during launch etc.

Likely model: https://www.ebad.com/tini-pin-puller/#pin-puler-p5

## Appendix: Motor selection
As of July 2022, we have been presuming usage of Globe Motor A-1430. This model indeed is offered with reduction ratios that deliver 300 oz-in of torque. Therefore we can proceed further with this motor family.

At the nominal operating voltage, the Globe Motors have two armature winding options:

In [39]:
motor_voltage = 12.  # VDC
globe_armature = {'-15': {'no load speed (rpm)': [13500, 17000],
                          'max rated torque (oz*in)': 0.22,
                          'theoretical stall (oz*in)': 2.60,
                          'Kt (oz*in/A)': 0.95,
                          'R': 3.70,  # ohms
                          'stall current': 3.2,  # amps
                         },
                  '-14': {'no load speed (rpm)': [10000, 13000],
                          'max rated torque (oz*in)': 0.33,
                          'theoretical stall (oz*in)': 2.00,
                          'Kt (oz*in/A)': 1.32,
                          'R': 6.46,  # ohms
                          'stall current': 1.9,  # amps
                         },
                 }

maxon_armature = {' DCX22S-12V': {'no load speed (rpm)': [12400],  # space character, and list type are for compatiblity with globe defs
                          'max rated torque (oz*in)': 14.6 / 1000 / Nm_per_ozin,
                          'theoretical stall (oz*in)': 108. / 1000 / Nm_per_ozin,
                          'Kt (oz*in/A)': 9.18 / 1000 / Nm_per_ozin,
                          'R': 1.02,  # ohms
                          'stall current': 11.8,  # amps
                         }
                 }

Among enclosed type motors, the family includes numerous options. The 12 gearhead configurations with max continous torque >= 45 oz\*in are tested below:

In [40]:
globe_gear = {'43A147': {'speed reduction ratio': 321,
                         'torque multiplier ratio': 130,
                         'length (in)': 3.11,
                         'max continuous torque (oz*in)': 45., 
                        },
              '43A148': {'speed reduction ratio': 485,
                         'torque multiplier ratio': 200,
                         'length (in)': 3.11,
                         'max continuous torque (oz*in)': 70., 
                        },
              '43A149': {'speed reduction ratio': 733,
                         'torque multiplier ratio': 300,
                         'length (in)': 3.11,
                         'max continuous torque (oz*in)': 100., 
                        },
              '43A150': {'speed reduction ratio': 1108,
                         'torque multiplier ratio': 450,
                         'length (in)': 3.11,
                         'max continuous torque (oz*in)': 150., 
                        },
              '43A151': {'speed reduction ratio': 1853,
                         'torque multiplier ratio': 600,
                         'length (in)': 3.28,
                         'max continuous torque (oz*in)': 200., 
                        },
              '43A152': {'speed reduction ratio': 2799,
                         'torque multiplier ratio': 900,
                         'length (in)': 3.28,
                         'max continuous torque (oz*in)': 300., 
                        },
              '43A153': {'speed reduction ratio': 4230,
                         'torque multiplier ratio': 1400,
                         'length (in)': 3.28,
                         'max continuous torque (oz*in)': 300., 
                        },
              '43A154': {'speed reduction ratio': 6391,
                         'torque multiplier ratio': 2100,
                         'length (in)': 3.28,
                         'max continuous torque (oz*in)': 300., 
                        },
              '43A155': {'speed reduction ratio': 10689,
                         'torque multiplier ratio': 2800,
                         'length (in)': 3.45,
                         'max continuous torque (oz*in)': 300., 
                        },
              '43A156': {'speed reduction ratio': 16150,
                         'torque multiplier ratio': 4200,
                         'length (in)': 3.45,
                         'max continuous torque (oz*in)': 300., 
                        },
              '43A157': {'speed reduction ratio': 24403,
                         'torque multiplier ratio': 6400,
                         'length (in)': 3.45,
                         'max continuous torque (oz*in)': 300., 
                        },
              '43A158': {'speed reduction ratio': 36873,
                         'torque multiplier ratio': 9700,
                         'length (in)': 3.45,
                         'max continuous torque (oz*in)': 300., 
                        },
             }

maxon_gear = {'GPX22UP-987:1': {'speed reduction ratio': 987,
                                 'torque multiplier ratio': 987 * 0.35, # approximate, based on analogy to Globe gears of similar speed reduction
                                 'length (in)': 52.0 / 25.4,
                                 'max continuous torque (oz*in)': 5.20 / Nm_per_ozin, 
                                },
              'GPX22UP-1526:1': {'speed reduction ratio': 1526,
                                 'torque multiplier ratio': 1526 * 0.35, # approximate, based on analogy to Globe gears of similar speed reduction
                                 'length (in)': 52.0 / 25.4,
                                 'max continuous torque (oz*in)': 5.20 / Nm_per_ozin, 
                                },
             }

Speed is calculated by a linear assumption for the torque/speed curve, i.e.:

$\omega_r \approx (1 - \frac{\tau_r}{\tau_o}) \omega_o$

where:

$~~~~\omega_r$ = angular speed of rotor

$~~~~\omega_o$ = no load speed

$~~~~\tau_r$ = torque of rotor

$~~~~\tau_o$ = stall torque

Electrical power consumed, mechanical power output, and efficiency of this conversion are estimated:

$P_{elec} \approx I V + I^2 R = \frac{\tau_r}{K_t} V + \left( \frac{\tau_r}{K_t} \right)^2 R$

$P_{mech} = \tau_g \omega_g \approx \frac{\beta_\tau}{\beta_\omega} \tau_r \omega_r$

$\eta = \frac{P_{mech}}{P_{elec}} = \frac{\beta_\tau}{\beta_\omega} \frac{K_t}{V} \omega_r$

where:

$~~~~I$ = current

$~~~~V$ = voltage

$~~~~R$ = resistance

$~~~~K_t$ = motor torque constant (N\*m/A or oz\*in/A)

$~~~~\tau_g$ = torque at gearhead output shaft

$~~~~\omega_g$ = angular speed at gearhead output shaft

$~~~~\beta_\tau = \frac{\tau_g}{\tau_r}$ = torque multiplier ratio of gearhead

$~~~~\beta_\omega = \frac{\omega_r}{\omega_g}$ = speed reduction ratio of gearhead

In [41]:
avg_harness_torque = harness_max_torque / 2
estim_loads = {'min continuous tau_g (oz*in)': (min(T_static_friction) + min(T_friction_transmission_loss) + avg_harness_torque) / Nm_per_ozin,
               'max continuous tau_g (oz*in)': (max(T_static_friction) + max(T_friction_transmission_loss) + avg_harness_torque) / Nm_per_ozin,
               'min intermittent tau_g (oz*in)': (T_known + min(T_variable)) / Nm_per_ozin,
               'max intermittent tau_g (oz*in)': (T_known + max(T_variable)) / Nm_per_ozin,
              }
gearmotor = []
for manufacturer_specs in [{'armature': globe_armature, 'gear': globe_gear},
                           {'armature': maxon_armature, 'gear': maxon_gear}]:
    for a, arm in manufacturer_specs['armature'].items():
        tau0_ozin = arm['theoretical stall (oz*in)']
        w0_rpm = max(arm['no load speed (rpm)'])  # pick one of the vendor values for simplicity
        Kt_ozinperA = arm['Kt (oz*in/A)']
        for g, gear in manufacturer_specs['gear'].items():
            gearmotor += [{'part': f'{g}{a}'}]
            gearmotor[-1]['arm'] = arm
            gearmotor[-1]['gear'] = gear
            beta_t = gear['torque multiplier ratio']
            beta_w = gear['speed reduction ratio']
            gearmotor[-1]['rated tau_r (oz*in)'] = arm['max rated torque (oz*in)']  # practical assumption
            gearmotor[-1]['rated tau_g (oz*in)'] = gearmotor[-1][f'rated tau_r (oz*in)'] * beta_t
            for prefix in ['rated'] + ['min continuous', 'max continuous', 'min intermittent', 'max intermittent']:
                tau_g_ozin = gearmotor[-1][f'{prefix} tau_g (oz*in)'] if prefix == 'rated' else estim_loads[f'{prefix} tau_g (oz*in)']
                tau_r_ozin = tau_g_ozin / beta_t
                tau_r = tau_r_ozin * Nm_per_ozin
                I = tau_r_ozin / Kt_ozinperA
                VI = I * motor_voltage
                w_r_rpm = (1 - tau_r_ozin / tau0_ozin) * w0_rpm
                w_r = w_r_rpm * radpersec_per_rpm
                gearmotor[-1][f'{prefix} tau_r (oz*in)'] = tau_r_ozin
                gearmotor[-1][f'{prefix} I'] = I
                gearmotor[-1][f'{prefix} P_elec'] = VI + I**2 * arm['R']
                gearmotor[-1][f'{prefix} P_mech'] = beta_t / beta_w * tau_r * w_r 
                gearmotor[-1][f'{prefix} w_r (rpm)'] = w_r_rpm
                gearmotor[-1][f'{prefix} w_g (rpm)'] = w_r_rpm / beta_w
                gearmotor[-1][f'{prefix} efficiency'] = gearmotor[-1][f'{prefix} P_mech'] / gearmotor[-1][f'{prefix} P_elec']
            gearmotor[-1]['gearhead continuous torque safety factor'] = gear['max continuous torque (oz*in)'] / gearmotor[-1]['rated tau_g (oz*in)']
            gearmotor[-1]['gearhead intermittent torque safety factor'] = 2 * gear['max continuous torque (oz*in)'] / estim_loads['max intermittent tau_g (oz*in)']
            gearmotor[-1]['output stall torque (oz*in)'] = tau0_ozin * beta_t

From this list, we select those motors which pass requirements on:
- continuous torque limit (per vendor datasheet)
- intermittent torque (per vendor datasheet)
- max and min design speeds (see section above)

In [42]:
torque_sf_tol = 0.1  # allow a bit of tolerance here for borderline cases, since in practice our continous torques are going to be much lower than rating
continuous_torque_ok, intermittent_torque_ok, max_speed_ok, min_speed_ok, bypass_selection = set(), set(), set(), set(), set()
for idx, g in enumerate(gearmotor):
    if g['gearhead continuous torque safety factor'] >= (1.0 - torque_sf_tol):
        continuous_torque_ok.add(idx)
    if g['gearhead intermittent torque safety factor'] >= (1.0 - torque_sf_tol):
        intermittent_torque_ok.add(idx)
    if g['max continuous w_g (rpm)'] * radpersec_per_rpm <= max_allowable_speed:
        max_speed_ok.add(idx)
    if g['min continuous w_g (rpm)'] * radpersec_per_rpm >= min_allowable_speed:
        min_speed_ok.add(idx)
passing_idxs = continuous_torque_ok & intermittent_torque_ok & max_speed_ok & min_speed_ok | bypass_selection
print(f'Found {len(passing_idxs)} motors meeting all selection requirements:')
for idx in sorted(passing_idxs):
    gm = gearmotor[idx]
    print(f'{gm["part"]}')

Found 8 motors meeting all selection requirements:
43A150-15
43A151-15
43A152-15
43A153-15
43A150-14
43A151-14
43A152-14
GPX22UP-987:1 DCX22S-12V


Of these, we are pursuing procurement on the following:

In [43]:
selected_parts = ['GPX22UP-987:1 DCX22S-12V']
selected_idxs = [i for i, gm in enumerate(gearmotor) if gm['part'] in selected_parts]
primary_gearmotor = gearmotor[selected_idxs[0]]  # one item selected, for use in downstream calculations

In [44]:
for idx in sorted(selected_idxs):
    gm = gearmotor[idx]
    print(f'\n{horiz_rule}\n')
    print(f'Part: {gm["part"]}')
    print(f'Speed reduction ratio = {gm["gear"]["speed reduction ratio"]}')
    tau0_g_ozin = gm['output stall torque (oz*in)']
    print(f'Stall torque at output shaft = {tau0_g_ozin*Nm_per_ozin:.1f} N*m = {tau0_g_ozin:.1f} oz*in = {tau0_g_ozin/16:.1f} in*lb')
    print(f'Stall current = {gm["arm"]["stall current"]:.2f} A')

    for kind in ['continuous', 'intermittent']:
        print(f'Gearhead {kind} torque safety factor = {gm[f"gearhead {kind} torque safety factor"]:.2f} (must be ~ 1 or greater)')
    for prefix, description in {'rated': 'Motor is rated for',
                         'min continuous': 'Estimated lows during continuous operation',
                         'max continuous': 'Estimated highs during continuous operation',
                         'min intermittent': 'Estimated lows during intermittent operation',
                         'max intermittent': 'Estimated highs during intermittent operation',
                        }.items():
        print('')
        print(f'{description}:')
        tau_g_ozin = gm[f"{prefix} tau_g (oz*in)"] if prefix == 'rated' else estim_loads[f'{prefix} tau_g (oz*in)']
        print(f'{tab}Output torque = {tau_g_ozin*Nm_per_ozin:.3f} N*m = {tau_g_ozin:.1f} oz*in = {tau_g_ozin/16:.1f} in*lb')
        print(f'{tab}Output speed = {gm[f"{prefix} w_g (rpm)"]:.3f} rpm = {math.degrees(gm[f"{prefix} w_g (rpm)"]*radpersec_per_rpm):.2f} deg/sec')
        print(f'{tab}Current = {gm[f"{prefix} I"]:.2f} A')
        print(f'{tab}Power consumption = {gm[f"{prefix} P_elec"]:.3f} W')
    
    for prefix in ['min', 'max']:
        print('')
        print(f'Max slew ({math.degrees(max_slew_distance):.1f}{deg}) at {prefix.upper()} speed:')
        w_g = gm[f'{prefix} continuous w_g (rpm)'] * radpersec_per_rpm
        continuous_tau_g = estim_loads[f'{prefix} continuous tau_g (oz*in)'] * Nm_per_ozin
        intermittent_tau_g = estim_loads[f'{prefix} intermittent tau_g (oz*in)'] * Nm_per_ozin
        accel_avg_tau_g = (continuous_tau_g + intermittent_tau_g) / 2
        accel_avg = accel_avg_tau_g / total_inertia
        accel_time =  w_g / accel_avg
        accel_energy = accel_time * gm[f'{prefix} intermittent P_elec']
        accel_distance = 0.5 * accel_avg * accel_time**2
        coast_distance = max_slew_distance - accel_distance  # ignoring decel distance here, which is conservative, and in line with the intended control scheme (i.e. cut power near final target)
        coast_time = coast_distance / w_g
        coast_energy = coast_time * gm[f'{prefix} continuous P_elec']
        total_est_time = coast_time + accel_time
        print(f'{tab}total time = {total_est_time:.1f} sec = {total_est_time/60:.1f} min')
        print(f'{tab}total energy = {coast_energy + accel_energy:.1f} J')


----------

Part: GPX22UP-987:1 DCX22S-12V
Speed reduction ratio = 987
Stall torque at output shaft = 37.3 N*m = 5283.3 oz*in = 330.2 in*lb
Stall current = 11.80 A
Gearhead continuous torque safety factor = 1.03 (must be ~ 1 or greater)
Gearhead intermittent torque safety factor = 19.08 (must be ~ 1 or greater)

Motor is rated for:
    Output torque = 5.044 N*m = 714.2 oz*in = 44.6 in*lb
    Output speed = 10.865 rpm = 65.19 deg/sec
    Current = 1.59 A
    Power consumption = 21.665 W

Estimated lows during continuous operation:
    Output torque = 0.080 N*m = 11.4 oz*in = 0.7 in*lb
    Output speed = 12.536 rpm = 75.22 deg/sec
    Current = 0.03 A
    Power consumption = 0.304 W

Estimated highs during continuous operation:
    Output torque = 0.312 N*m = 44.2 oz*in = 2.8 in*lb
    Output speed = 12.458 rpm = 74.75 deg/sec
    Current = 0.10 A
    Power consumption = 1.192 W

Estimated lows during intermittent operation:
    Output torque = 0.277 N*m = 39.2 oz*in = 2.4 in*lb
    Out

### Torque limiting plan
There are three protections built into the system against failures due to over-torquing:

- In nominal operation we never hit the hardstop, staying well clear of it with encoder readings.
- Hardstops will be located immediately near the motor output, so load can be reacted locally (i.e. not transmit through rest of the drivetrain).
- Our current monitor reading will allow us to cut power to motor before the torque can ramp up. This protects against two different fault cases:
   - inadvertent hardstop strike (encoder problem)
   - bearing seizes up (mechanical problem)

We have not deeply investigated the option of including a mechanical torque limiting device, due to not knowing of a space-rated, vacuum compatible model appropriate to our application. Such a device may well exist, but we have not as of this writing (Sept 2022) identified one.

In detail, the plan and its rationale are as follows (reproduced from an email thread Sept 30, 2022 among the engineering team):

1. Carousel will have hard mechanical limits on travel at $\sim~\pm 100^o$.
2. Encoder is absolute.
3. Motor is same model as used in Mars 2020 rover, thus alleviating significant cost and time of qualification for our small project. Quite lightweight, though really somewhat more powerful than we really need.
4. High gear ratios chosen so as to inherently limit the output speed. At 987:1 reduction ratio with our nominal friction loads, I calculate ~ 75 deg/sec top speed. At 1526:1 (max offered by vendor) expect ~ 49 deg/sec.
5. Our expected torque loads are ~ 0.1 - 0.4 N\*m, depending on friction estimate. For which motor current ~ 30 - 120 mA
6. Motor stall current = 11.8 A, inrush maybe ~ 16.5 A or so.
7. Theoretical stall torque at gear output is massive at these gear ratios, ~ 37 N\*m (987:1 gear) or 58 N\*m (1526:1).
8. When hitting hard limit, full stall torque would require massively overbuilt drive train components. The high torque would also have to react through the top plate structure all the way through LuSEE to the base plate. This would all have to be very heavy, thus undesirable.
9. Instead, proposing two layers of fault protection:
   9a. Software position limits at ~ +/- 95 deg (measured by absolute encoder).
   9b. Current sensor (see point 11 below).
10. If either (9a) or (9b) exceeds its respective threshold, immediately cut power (i.e. open solid-state switch) to motor.
11. Suggest likely current threshold: ~ 90% of rated continuous current (1.65 A) = 1.5 A, when sustained for more than ~ 5-10 inrush time constants (~ 0.3 - 0.6 ms).
12. Mechanical spin up time constant for motor (not including gearhead, thus conservative) is ~ 6.1 ms, therefore the threshold duration proposed in item (9) should prevent any significant torque build up.
13.  During testing in the lab, we could perhaps incorporate a temporary pin that breaks at low torque, until we are confident in our current monitor and thresholds.

## Appendix: Bearings selection

### Bearing type and size
The preliminary design (July 2022) packaging acommodates a pair of Kaydon KA025AR0 bearings:

![Kaydon KA0xxAR0](figures/kaydon-KA0xxAR0-open-A-bearings.png)

This is a thin section, angular contact, open bearing, with bore 2.5\" (63.5 mm), O.D. 3.0\" (76.2 mm), and thickness 0.25\" (6.35 mm). Mass is low, ~0.12 lbs = 55 g. We utilize the common design pattern of mounting in pairs, to provide tilt resistance.

Stainless steel options (may have custom lead-times and will be more expensive) are available. Replace "K" with "S" in the part number.

Kaydon also offers an attractive "Type X" four-point contact bearing. We have previously used such a bearing (at a much larger scale) in the Dark Energy Spectroscopic Instrument (DESI) ADC rotator mechanism. The type X controls 5 degrees of freedom in a single unit, rather than the typical angular contact duplex. This has the advantage of fewer parts and less weight, as well as removing the need to control thrust preload in the mounting between the two separate bearing units. A type X unit of similar package size as KA025AR0 would be KA025XP0:

![Kaydon KA0xxXP0](figures/kaydon-KA0xxXP0-open-X-bearings.png)

Again, replace "K" with "S" for equivalent stainless option.

### External vs internal bearing seals

For these open bearings, we would make a dust seal that rides on the shaft and/or flat face. Call this an "external" seal. It has the advantages of being:

- independent from the bearing selection and procurement
- easily replaceable
- visible and inspectable
- material of our choice (i.e. robust, low cte, temperature tolerant material like polyimide)

We can alternatively purchase a bearing with integrated seals. Call this an "internal" seal. It has the advantages of:

- lower part count
- lower mass
- more compact packaging

Again taking Kaydon as an example bearing manufacturer, their standard seal material is nitrile rubber. This would need to be qualified for our temperature range (100 K - 400 K). Kaydon does offer customization options in alternate seal materials, but we have not yet checked whether polyimide in particular would work for them. Several models Kaydon offers with an integrated double-seal are given in this table:

![Kaydon JA0xxXP0](figures/kaydon-JA0xxXP0-sealed-X-bearings.png)

The Kaydon line-up does not have sealed bearings in angular contact configurations. 

For sealed bearings, replace "J" with "W" in the part number for the stainless option.

### Bearing seal materials

If we fabricate our own seals, one might consider using [Vespel SP-3](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-3). This is a 15% MoS<sub>2</sub>-filled polyimide with a long history of use in space applications, with low creep, high strength, and low coefficient of thermal expansion (for a polymer), and which should survive the temperature extremes.

A concern I have is that the friction coefficient of SP-3 in vacuum is reported to be very low (0.03) in the Dupont datasheet. Normally lower friction is "good" in a bearing, however in our design, having some consistent friction is helpful, to guarantee stability of the position of the turntable stable when powered off. So we probably rather prefer having a consistent friction whether testing in lab air or operating in vacuum. In this respect, unfilled [Vespel SP-1](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-1) may be a simpler and better choice. However, the DuPont datasheet does not specifically list a vacuum friction coefficient, so this bears some further investigation.

Graphite-filled polyimide grades would not be chosen, since the graphite lubricant can in fact become abrasive in a moisture-free environment.

### Turntable bearings

There are some bearings available with more integrated turntable functionality, for example including an external gear profile. An attractive option may be Kaydon T01-00325EAA:

![Kaydon turntables - basic](figures/kaydon-turntable-bearings-basic.png)

![Kaydon turntables - external gears](figures/kaydon-turntable-bearings-extgear.png)

![Kaydon turntables - illustrations](figures/kaydon-turntable-illustrations.png)


These turntables are not off-the-shelf parts. Cost and lead-time would be TBD. Performance is not particularly any better than the other options; the attraction here would be the compact packaging. The gearing would need some exterior protection from lunar dust.

In [45]:
bearing = {'JA025XP0':
               {'o.d. (in)': 3.0,
                'i.d. (in)': 2.5,
                'weight (lb)': 0.12,
                'dynamic radial (lbf)': 583,
                'dynamic thrust (lbf)': 910,
                'dynamic moment (in*lbf)': 601,
                'static radial (lbf)': 830,
                'static thrust (lbf)': 2090,
                'static moment (in*lbf)': 1150,
                },
           'JA030XP0':
               {'o.d. (in)': 3.5,
                'i.d. (in)': 3.0,
                'weight (lb)': 0.14,
                'dynamic radial (lbf)': 643,
                'dynamic thrust (lbf)': 1010,
                'dynamic moment (in*lbf)': 785,
                'static radial (lbf)': 990,
                'static thrust (lbf)': 2470,
                'static moment (in*lbf)': 1600,
                },
           'JA035XP0':
               {'o.d. (in)': 4.0,
                'i.d. (in)': 3.5,
                'weight (lb)': 0.17,
                'dynamic radial (lbf)': 701,
                'dynamic thrust (lbf)': 1110,
                'dynamic moment (in*lbf)': 986,
                'static radial (lbf)': 1140,
                'static thrust (lbf)': 2850,
                'static moment (in*lbf)': 2130,
                },
           'T01-00325EAA':
               {'o.d. (in)': 4.078,
                'i.d. (in)': 2.5,
                'weight (lb)': 0.50,
                'dynamic radial (lbf)': 640,
                'dynamic thrust (lbf)': 1010,
                'dynamic moment (in*lbf)': 780,
                'static radial (lbf)': 990,
                'static thrust (lbf)': 2470,
                'static moment (in*lbf)': 1600,
                },
           'T01-00375EAA':
               {'o.d. (in)': 4.578,
                'i.d. (in)': 3.0,
                'weight (lb)': 0.59,
                'dynamic radial (lbf)': 700,
                'dynamic thrust (lbf)': 1110,
                'dynamic moment (in*lbf)': 980,
                'static radial (lbf)': 1140,
                'static thrust (lbf)': 2850,
                'static moment (in*lbf)': 2130,
                },
          }

print(bearing_loads_str, '\n')
print(f'This is for a turntable plate in {plate_material}, of diameter {plate_od:.3f} m.\n')
print(f'Margins on the above loads for specific bearing models are\ngiven below. These must all be >= 0 for a given model.\n')
for name, data in bearing.items():
    print(f'Bearing model {name} (mass = {data["weight (lb)"]*N_per_lbf/G_earth:.3f} kg),\nmargins w.r.t. design loads:')
    for mode in ['rms', 'peak']:
        for direction in ['radial', 'thrust', 'moment']:
            key = f'static {direction} '
            if direction == 'moment':
                key += '(in*lbf)'
                conversion = Nm_per_inlb
            else:
                key += '(lbf)'
                conversion = N_per_lbf
            margin = data[key] * conversion / bearing_design_loads[mode][direction] - 1
            print(f'{tab}{mode} {direction} = {margin:.2f}')
    print('')
                        

Payload mass of 4.51 kg yields the following bearing
design loads (these include all safety factors):
    rms radial = 936 N = 210 lbf
    rms thrust = 936 N = 210 lbf
    rms moment = 55 N*m = 483 in*lb
    peak radial = 4681 N = 1052 lbf
    peak thrust = 4681 N = 1052 lbf
    peak moment = 273 N*m = 2413 in*lb 

This is for a turntable plate in aluminum, of diameter 0.400 m.

Margins on the above loads for specific bearing models are
given below. These must all be >= 0 for a given model.

Bearing model JA025XP0 (mass = 0.054 kg),
margins w.r.t. design loads:
    rms radial = 2.94
    rms thrust = 8.93
    rms moment = 1.38
    peak radial = -0.21
    peak thrust = 0.99
    peak moment = -0.52

Bearing model JA030XP0 (mass = 0.063 kg),
margins w.r.t. design loads:
    rms radial = 3.70
    rms thrust = 10.73
    rms moment = 2.32
    peak radial = -0.06
    peak thrust = 1.35
    peak moment = -0.34

Bearing model JA035XP0 (mass = 0.077 kg),
margins w.r.t. design loads:
    rms radia

Among these, model JA035XP0 (or WA035XP0 for stainless version) is selected for comfortably meeting all safety factors. At 77 g, it is quite mass efficient (compare to duplexed pair of KA025AR0 at 55 g each x 2 = 110 g). It additionally has the benefit of a wider inner bore (3.5", compare to 2.5" for KA025AR0) through which to route the antenna cables and communications mast.

## References
- [NASA General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf)
- [Kaydon turntable bearings](https://www.kaydonbearings.com/RealiSlim_TT_bearings.htm)
- [Kaydon sealed, slim bearings](https://www.kaydonbearings.com/RealiSlim_sealed_bearings.htm)